In [4]:
from copy import deepcopy
import random
from models import *
import yaml
import os
import numpy as np
import datetime as dt
import time
import pandas as pd
from helpers import colorize, create_node_matrix
from warnings import filterwarnings

filterwarnings('ignore')

data_filename = "HConVRPDatasets_YML/Medium/15%/b1.yml" 

data = yaml.load(open(data_filename), Loader=yaml.FullLoader)

depot = Customer(0, tuple(data["Depot_coordinates"]), [0]*data["Planning_Horizon"], data["Planning_Horizon"])

def initialize_customers(data, planning_horizon):
    customers = []
    for idx, coordinates in enumerate(data["Customer_coordinates"]):
        coordinates = tuple(coordinates)
        demands = data["Customer_demands"][idx]
        new_cust = Customer(idx + 1, coordinates, demands, planning_horizon)
        customers.append(new_cust)
    return customers

def initialize_vehicle_types(data):
    vehicle_types = []
    for idx, vehicle_type in enumerate(data["Vehicle_Types"]):
        vehicle_type_name = vehicle_type["Vehicle_Type"]
        capacity = vehicle_type["Capacity"]
        fixed_cost = vehicle_type["Fixed_Cost"]
        variable_cost = vehicle_type["Variable_Cost"]
        speed = vehicle_type["Speed"]
        available_vehicles = vehicle_type["Number_of_available_vehicles"]
        new_v_type = VehicleType(vehicle_type_name, available_vehicles, capacity, fixed_cost, variable_cost, speed)
        vehicle_types.append(new_v_type)
    return vehicle_types

def initialize_vehicles(data, vehicle_types, customers, distance_matrix):
    compatability_matrix = data["Compatibility_Restrictions"]
    compatability_matrix = np.array(compatability_matrix)

    planning_horizon = data["Planning_Horizon"]
    route_duration = data["Route_Duration"]

    vehicles = []
    for vehicle_type in vehicle_types:
        for idx in range(vehicle_type.available_vehicles):
            compatible_customers = []
            incompatible_customers = []
            if compatability_matrix.size > 0:
                for customer in customers:
                    if compatability_matrix[customer.id - 1][len(vehicles) + 1] == 1:
                        compatible_customers.append(customer)
                    else:
                        incompatible_customers.append(customer)
            else:
                compatible_customers = customers

            new_vehicle = Vehicle(len(vehicles) + 1, vehicle_type, depot,
                                  planning_horizon, route_duration, distance_matrix,
                                  compatible_customers, incompatible_customers)
            vehicles.append(new_vehicle)
    return vehicles

planning_horizon = data["Planning_Horizon"]
route_duration = data["Route_Duration"]

customers = initialize_customers(data, planning_horizon)
vehicle_types = initialize_vehicle_types(data)
distance_matrix = create_node_matrix(customers, depot, type_matrix="distance")

vehicles = initialize_vehicles(data, vehicle_types, customers, distance_matrix)

print("Number of customers:", colorize(len(customers), 'GREEN'))
print("Number of vehicle types:", colorize(len(vehicle_types), 'GREEN'))
print("Number of vehicles:", colorize(len(vehicles), 'GREEN'))
print("Planning horizon:", colorize(planning_horizon, 'YELLOW'))
print("Route duration:", colorize(route_duration, 'YELLOW'))
print("-"*50)

Number of customers: 50
Number of vehicle types: 3
Number of vehicles: 4
Planning horizon: 5
Route duration: 240
--------------------------------------------------


In [8]:
customers[0] in vehicles[1].compatible_customers

False